# Exploration des données

In [ ]:

import numpy as np #  manip des arrays (tableaux)
import pandas as pd #  manip des dataframes
import matplotlib.pyplot as plt #  construct des graphes : barplot, cammenberts
import seaborn as sns

df = pd.read_csv('processed.cleveland.csv', names=['age','sex','chest pain','blood pressure(r)','cholestoral','fasting blood sugar','eleccardio results(r)','max heart rate achvd','exercise angina','ST depression (ex to r)','slope exercise ST','number of vessels','thalassemia','diagnosis'])
df.describe()

df.head()
df.describe()

In [ ]:
df.boxplot(figsize=[8,6], rot=70)

In [ ]:
# from pandas.plotting import scatter_matrix
# scatter_mat = scatter_matrix(df, figsize = (20,16))

axes = pd.plotting.scatter_matrix(df, alpha=0.2, figsize = (16,12))
for ax in axes.flatten():
    ax.xaxis.label.set_rotation(90)
    ax.yaxis.label.set_rotation(0)
    ax.yaxis.label.set_ha('right')

plt.tight_layout()
plt.gcf().subplots_adjust(wspace=0, hspace=0)
plt.show()

In [ ]:
df_corr = df.corr()
df_corr

In [ ]:
cmap = cmap=sns.diverging_palette(5, 250, as_cmap=True)

def magnify():
    return [dict(selector="th",
            props=[("font-size", "7pt")]),
            dict(selector="td",
            props=[('padding', "0em 0em")]),
            dict(selector="th:hover",
            props=[("font-size", "12pt")]),
            dict(selector="tr:hover td:hover",
            props=[('max-width', '200px'),
            ('font-size', '12pt')])
        ]
        
df_corr.style.background_gradient(cmap, axis=1).set_properties(**{'hello':'80px','font-size':'10pt'}).set_caption("Hover to magify").set_precision(2).set_table_styles(magnify())



In [ ]:
# Régression linéaire ajustée au graphique précédent
sns.regplot(x=df["max heart rate achvd"], y=df["age"], fit_reg=True)
plt.show()

In [ ]:
# df.plot(x='exercise angina', y='chest pain', style='o')
sns.regplot(x=df["exercise angina"], y=df["chest pain"], fit_reg=True)
plt.show()

In [ ]:
# df.plot(x='exercise angina', y='chest pain', style='o')
sns.regplot(x=df["cholestoral"], y=df["diagnosis"], fit_reg=True)
plt.show()

In [ ]:
# df.plot(x='blood pressure(r)', y='cholestoral', style='o')
sns.regplot(x=df["blood pressure(r)"], y=df["cholestoral"], fit_reg=True)
plt.show()


In [ ]:
graph = df.groupby(['age'])['age'].count()
graph.plot(kind = 'bar', figsize=(16, 10))

In [ ]:
graph = df.groupby(['sex'])['sex'].count()
graph.plot(kind = 'bar', figsize=(16, 10))

In [ ]:
# On nettoie les valeurs nulles
df = df[df["number of vessels"].str.contains("\?")==False]
df = df[df["thalassemia"].str.contains("\?")==False]

df.describe()


In [ ]:
from sklearn.preprocessing import MinMaxScaler

min_max = MinMaxScaler()

df_minmax = min_max.fit_transform(df)
df_minmax = pd.DataFrame(df_minmax)
df_minmax.boxplot(rot = 45,figsize=(16, 10))

In [ ]:
from sklearn.preprocessing import scale

df_scale = scale(df)
df_scale
pd.DataFrame(df_scale).boxplot(rot = 45,figsize=(16, 10))

# Split

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# take only most important columns
X = df_minmax.iloc[:,[0,2,7,8,9,10]]
# X = df_minmax.iloc[:,0:13]
y = df_minmax.iloc[:,-1]

# print(X)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=1/4, random_state=42, stratify=y)

# FIXME : Restore initial classes
le = LabelEncoder()
le.fit(y_train)
y_train = le.transform(y_train)
le.fit(y_test)
y_test = le.transform(y_test)

# Entraîner le modèle

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(n_neighbors=1)
knn.fit(X_train, y_train)
knn.effective_metric_

# Prédir

In [ ]:
y_pred = knn.predict(X_test)
 

# Evaluer

In [ ]:
from sklearn import metrics

def evaluate_this(y_test,y_pred):
    print(metrics.accuracy_score(y_test, y_pred))
    print(metrics.f1_score(y_test, y_pred, average='weighted'))

evaluate_this(y_test,y_pred)

# Try to guess better K

## k-fold cross validation

In [ ]:
# https://towardsdatascience.com/how-to-find-the-optimal-value-of-k-in-knn-35d936e554eb pour l'approche basique
# https://kevinzakka.github.io/2016/07/13/k-nearest-neighbor/ pour le cross validation

from sklearn.model_selection import cross_val_score

# creating odd list of K for KNN
neighbors = list(range(1, 50, 2))

cv_scores = []
error_rate = []

for k in neighbors:
    knn = KNeighborsClassifier(n_neighbors=k)
    scores = cross_val_score(knn, X_train, y_train, cv=10, scoring='accuracy')
    cv_scores.append(scores.mean())
    
    knn.fit(X_train, y_train)
    y_pred = knn.predict(X_test)
    error_rate.append(np.mean(y_pred != y_test))

# changing CV accuracy => to CV error
cv_errors = [1 - x for x in cv_scores]

# determining best k for both methods
cv_optimal_k = neighbors[cv_errors.index(min(cv_errors))]
error_optimal_k = neighbors[error_rate.index(min(error_rate))]

plt.figure(figsize=(10,6))
plt.plot(neighbors, cv_errors, color='green', linestyle='dashed', 
         marker='o', markersize=6)

plt.plot(neighbors, error_rate, color='grey', linestyle='dashed', 
         marker='o', markersize=6)


plt.title('CV errors AND Erros VS K')
plt.xlabel("K")
plt.ylabel("Error rate")

print("Minimum CV error : ", min(cv_errors)," % at K =", cv_optimal_k)
print("Minimum error: ", min(error_rate)," % at K =", error_optimal_k)


### Best K is 17 !

> 5 seems good, but it is probably overfiting

In [ ]:
df = pd.DataFrame({"neighbors" : neighbors, "CV errors" : cv_errors, "Errors" : error_rate}, index=neighbors)
# df = df.sort_values('errors', ascending=True)
df = df.sort_values(['CV errors', 'neighbors'], ascending=[True, False])
df.head(15)

# Rerun model with new K

In [ ]:
K = 15

knn = KNeighborsClassifier(n_neighbors=K)
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)

evaluate_this(y_test, y_pred)

# Decision boudaries

## First -- bad -- method : take multiple features

In [ ]:
from matplotlib.colors import ListedColormap
from sklearn import neighbors, datasets
import sys

h = .02  # step size in the mesh

X_new = X.values[:, [0,4]]
le.fit(y.values)
y_new = le.transform(y.values)

# Create color maps
cmap_light = ListedColormap(['#FFAAAA', '#AAFFAA', '#AAAAFF'])
cmap_bold = ListedColormap(['#FF0000', '#00FF00', '#0000FF'])

for weights in ['uniform', 'distance']:
    # we create an instance of Neighbours Classifier and fit the data.
    knn = KNeighborsClassifier(n_neighbors = K, weights = weights)
    knn.fit(X_new, y_new)

    # Plot the decision boundary. For that, we will assign a color to each
    # point in the mesh [x_min, x_max]x[y_min, y_max].
    x_min, x_max = X_new[:, 0].min() - 1, X_new[:, 0].max() + 1
    y_min, y_max = X_new[:, 1].min() - 1, X_new[:, 1].max() + 1

    xx, yy = np.meshgrid(np.arange(x_min, x_max, h),
                         np.arange(y_min, y_max, h))

    Z = knn.predict(np.c_[xx.ravel(), yy.ravel()])

    # Put the result into a color plot
    Z = Z.reshape(xx.shape)
    plt.figure()
    plt.pcolormesh(xx, yy, Z, cmap=cmap_light)

    # Plot also the training points
    plt.scatter(X_new[:, 0], X_new[:, 1], c=y, cmap=cmap_bold)
    plt.xlim(xx.min(), xx.max())
    plt.ylim(yy.min(), yy.max())
    plt.title("3-Class classification (k = %i, weights = '%s')"
              % (K, weights))

plt.show()

## Instead, find out what are the 2 most important features

### For that, plot feature density

In [ ]:
# All credits go to https://stackoverflow.com/questions/56153726/plot-k-nearest-neighbor-graph-with-8-features 

import pandas as pd
from pylab import rcParams
import matplotlib.pyplot as plt
from operator import itemgetter

def plot_densities(data):
    '''
    Plot features densities depending on the outcome values
    '''
    # change fig size to fit all subplots beautifully 
    rcParams['figure.figsize'] = 15, 20

    # separate data based on outcome values 
    outcome_0 = data[data['diagnosis'] == 0]
    # outcome_1 = data[data['diagnosis'] == 1]
    outcome_2 = data[data['diagnosis'] == 2]
    # outcome_3 = data[data['diagnosis'] == 3]
    outcome_4 = data[data['diagnosis'] == 4]

    # init figure
    fig, axs = plt.subplots(6, 1)
    fig.suptitle('Features densities for different outcomes 0/1')
    plt.subplots_adjust(left = 0.25, right = 0.9, bottom = 0.1, top = 0.95,
                        wspace = 0.2, hspace = 0.9)
    count = 0
# 0,2,7,8,9,10
    # plot densities for outcomes
    # for column_name in names[0,2,7,8,9,10]: 
    for column_name in itemgetter(0,2,7,8,9,10)(names): 

        ax = axs[count]
        #plt.subplot(4, 2, names.index(column_name) + 1)
        outcome_0[column_name].plot(kind='density', ax=ax, subplots=True, 
                                    sharex=False, color="blue", legend=True,
                                    label=column_name + ' for Outcome = 0')
        outcome_2[column_name].plot(kind='density', ax=ax, subplots=True, 
                                     sharex=False, color="green", legend=True,
                                     label=column_name + ' for Outcome = 2')
        outcome_4[column_name].plot(kind='density', ax=ax, subplots=True, 
                                     sharex=False, color="red", legend=True,
                                     label=column_name + ' for Outcome = 4')
        ax.set_xlabel(column_name + ' values')
        ax.set_title(column_name + ' density')
        ax.grid('on')
        count = count + 1
    plt.show()
    fig.savefig('densities.png')

# load your data 
data = pd.read_csv('processed.cleveland.csv', names=['age','sex','chest pain','blood pressure(r)','cholestoral','fasting blood sugar','eleccardio results(r)','max heart rate achvd','exercise angina','ST depression (ex to r)','slope exercise ST','number of vessels','thalassemia','diagnosis'])
names = list(data.columns)

plot_densities(data)

> Exercice angina and ST Depression are the more important features !

> Furthermore, the correlation between these two is relatively low (0.29; check correlation graph), which means they are not similar. 

### You can now draw decision boundarie graph

> Use the two most important features !

In [ ]:
# All credits go to https://stackoverflow.com/questions/56153726/plot-k-nearest-neighbor-graph-with-8-features 

import warnings 
import numpy as np
import pandas as pd
from pylab import rcParams
import matplotlib.pyplot as plt
from sklearn import neighbors
from matplotlib.colors import ListedColormap
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import scale
# filter warnings
warnings.filterwarnings("ignore")

def classify_and_plot(X, y):
    ''' 
    split data, fit, classify, plot and evaluate results 
    '''
    # split data into training and testing set
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.33, random_state = 41)

    # init vars
    n_neighbors = 1
    h           = .02  # step size in the mesh

    # Create color maps
    cmap_light = ListedColormap(['#FFAAAA', '#AAAAFF'])
    cmap_bold  = ListedColormap(['#FF0000', '#0000FF'])

    rcParams['figure.figsize'] = 5, 5
    for weights in ['uniform', 'distance']:
        # we create an instance of Neighbours Classifier and fit the data.
        clf = neighbors.KNeighborsClassifier(n_neighbors, weights=weights)
        clf.fit(X_train, y_train)

        # Plot the decision boundary. For that, we will assign a color to each
        # point in the mesh [x_min, x_max]x[y_min, y_max].
        x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1
        y_min, y_max = X[:, 1].min() - 1, X[:, 1].max() + 1
        xx, yy = np.meshgrid(np.arange(x_min, x_max, h),
                             np.arange(y_min, y_max, h))
        Z = clf.predict(np.c_[xx.ravel(), yy.ravel()])

        # Put the result into a color plot
        Z = Z.reshape(xx.shape)
        fig = plt.figure()
        plt.pcolormesh(xx, yy, Z, cmap=cmap_light)

        # Plot also the training points, x-axis = 'Glucose', y-axis = "BMI"
        plt.scatter(X[:, 0], X[:, 1], c=y, cmap=cmap_bold, edgecolor='k', s=20)   
        plt.xlim(xx.min(), xx.max())
        plt.ylim(yy.min(), yy.max())
        plt.title("K = %i, weights = '%s')" % (n_neighbors, weights))
        plt.show()
        fig.savefig(weights +'.png')

data = pd.read_csv('processed.cleveland.csv', names=['age','sex','chest pain','blood pressure(r)','cholestoral','fasting blood sugar','eleccardio results(r)','max heart rate achvd','exercise angina','ST depression (ex to r)','slope exercise ST','number of vessels','thalassemia','diagnosis'])

names = list(data.columns)

# we only take the best two features and prepare them for the KNN classifier
# 3 and 9 are the choosen features
X_prime  = np.array(data.iloc[:, [3,9]])
# scale X data
X        = scale(X_prime)
y        = np.array(data.iloc[:,-1])

# classify, evaluate and plot results
classify_and_plot(X, y)

# Construction de l'arbre

In [ ]:
from sklearn.tree import DecisionTreeClassifier
tree = DecisionTreeClassifier(random_state=42)
tree.fit(X_train, y_train)

y_pred = tree.predict(X_test)
y_pred

In [ ]:
from sklearn.metrics import confusion_matrix
mat_conf = confusion_matrix(y_test, y_pred)
mat_conf

In [ ]:
evaluate_this(y_test, y_pred)

In [ ]:
from sklearn.tree import export_text
tree_rules = export_text(tree, feature_names=X_train.columns.values.tolist())
# tree_rules

In [ ]:
scores=[]
for max_depth in range(2, 10) : 
    tree=DecisionTreeClassifier(max_depth=max_depth, random_state=42)
    tree.fit(X_train, y_train)
    score=tree.score(X_test, y_test)
    scores.append(score)
plt.plot(scores)

In [ ]:
# Algo optimal pr les besoins du plot de l'arbre
tree = DecisionTreeClassifier(max_depth=4, random_state=42)
tree.fit(X_train, y_train)

from sklearn.metrics import accuracy_score
acc = accuracy_score(y_test, y_pred)
acc

In [ ]:
from sklearn.tree import plot_tree
plt.figure(figsize = (10, 10))
tree_heart_disease = plot_tree(tree)

In [ ]:
tree_rules = export_text(tree, feature_names=X_train.columns.values.tolist())
# tree_rules